# Image Figures

This notebook is used to visualize the results from data fusion/gap-filling process.


In [ ]:
import ee
import geemap
from geemap import cartoee
import cartopy.crs as ccrs
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl


In [ ]:
# authenticate with Earth Engine
ee.Initialize(
    project='byu-hydroinformatics-gee'
)

## Gabon case

In [ ]:
# specify the region 
region = [9.041135,-1.856765, 9.931033,-0.95823]

In [ ]:
# load in the exported collection
ic = ee.ImageCollection('projects/byu-hydroinformatics-gee/assets/kmarkert/datafusion/Gabon_data_fusion') \
    .filterBounds(ee.Geometry.Rectangle(region))

In [ ]:
# filter for the SAR images
sar = ic.filter(ee.Filter.stringStartsWith('system:index', 'S1'))

In [ ]:
# filter for the optical images
optical = ic.filter(ee.Filter.stringStartsWith('system:index', 'LC08'))

In [ ]:
# specify the time range for data to visualize
start_time = "2019-05-16"
end_time = "2019-05-30"

In [ ]:
# get the SAR and optical images for the time range
sar_water = sar.filterDate(start_time, end_time).first()
optical_water = optical.filterDate(start_time, end_time).first()

In [ ]:
# get the date info for the observations
sar_obs_date = sar_water.date().format('YYYY-MM-dd').getInfo()
optical_obs_date = optical_water.date().format('YYYY-MM-dd').getInfo()

In [ ]:
# check to make sure that there is an optical image
optical_water.getInfo() is None

In [ ]:
# get the raw observations for visualization
sentinel1 = ee.ImageCollection("COPERNICUS/S1_GRD");
sentinel1_img = sentinel1.filter(ee.Filter.eq('system:index', sar_water.get('system:index'))).first()
s1_vis = {"bands": 'VV', "min":-25, "max":0};

landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2");
landsat8_img = landsat8.filter(ee.Filter.eq('system:index', optical_water.get('system:index'))).first()
lc8_vis = {"bands":'SR_B7,SR_B5,SR_B3',"gamma":1.5, "min": 7500 , "max":30000};


In [ ]:
fig,ax = plt.subplots(ncols=3,nrows=2,figsize=(15,10),
                      subplot_kw={'projection': ccrs.PlateCarree()})

dimsize = 1500

# use cartoee to get a map
cartoee.add_layer(ax[0,0], sentinel1_img, region=region, vis_params=s1_vis, dims=dimsize)
cartoee.add_layer(ax[0,1], sar_water, region=region, vis_params={"bands":'proba',"min":0,"max":100}, cmap='magma', dims=dimsize)
cartoee.add_layer(ax[0,2], sar_water, region=region, vis_params={"bands":'water',"min":0,"max":1}, cmap='Blues', dims=dimsize)


# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax[0,0], interval=[0.25, 0.25], linestyle=":")
cartoee.add_gridlines(ax[0,1], interval=[0.25, 0.25], linestyle=":")
cartoee.add_gridlines(ax[0,2], interval=[0.25, 0.25], linestyle=":")


ax[0,1].get_yaxis().set_visible(False)
ax[0,2].yaxis.tick_right()

ax[0,0].set_title(f'a) Sentinel 1 observation {sar_obs_date}',loc='left', fontsize=10)
ax[0,1].set_title(f'b) Sentinel 1 water probability',loc='left', fontsize=10)
ax[0,2].set_title(f'c) Sentinel 1 predicted water',loc='left', fontsize=10)



# use cartoee to get a map
cartoee.add_layer(ax[1,0], landsat8_img, region=region, vis_params=lc8_vis, dims=dimsize)
cartoee.add_layer(ax[1,1], optical_water, region=region, vis_params={"bands":'proba',"min":0,"max":100}, cmap='magma', dims=dimsize)
cartoee.add_layer(ax[1,2], optical_water, region=region, vis_params={"bands":'water',"min":0,"max":1}, cmap='Blues', dims=dimsize)


# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax[1,0], interval=[0.25, 0.25], linestyle=":")
cartoee.add_gridlines(ax[1,1], interval=[0.25, 0.25], linestyle=":")
cartoee.add_gridlines(ax[1,2], interval=[0.25, 0.25], linestyle=":")


ax[1,1].get_yaxis().set_visible(False)
ax[1,2].yaxis.tick_right()


ax[1,0].set_title(f'd) Landsat 8 observation {optical_obs_date}',loc='left', fontsize=10)
ax[1,1].set_title(f'e) Landsat 8 water probability',loc='left', fontsize=10)
ax[1,2].set_title(f'f) Landsat 8 predicted water (gap filled)',loc='left', fontsize=10)


cax1 = fig.add_axes([0.41, 0.05, 0.2, 0.007])
cax2 = fig.add_axes([0.69, 0.05, 0.2, 0.007])

norm1 = mpl.colors.Normalize(vmin=0, vmax=100)

cb1 = mpl.colorbar.ColorbarBase(cax1, cmap=mpl.cm.magma,
                                norm=norm1,
                                orientation='horizontal')
cb1.set_label('Probability [%]')

bounds = [0.0, 0.5, 1.0]
cmap = mpl.colormaps['Blues']
cbarcolors = mpl.colors.ListedColormap([mpl.colors.rgb2hex(cmap(i)) for i in range(2)])
norm2 = mpl.colors.BoundaryNorm(bounds, cmap.N)

cb2 = mpl.colorbar.ColorbarBase(cax2, cmap=cmap,
                                norm=norm2,
                                orientation='horizontal')

cb2.ax.set_xticks([0.25,0.75])
cb2.ax.set_xticklabels(['No water','Water'])

plt.savefig('gabon_sufacewater.png',dpi=300,bbox_inches='tight',facecolor='white')

plt.show()

## Cambodia flood case

In [ ]:
region = [103.9, 12.0, 105.2054, 12.75]

In [ ]:
ic = ee.ImageCollection('projects/byu-hydroinformatics-gee/assets/kmarkert/datafusion/Cambodia_data_fusion_v4') \
    .filterBounds(ee.Geometry.Rectangle(region).centroid())

In [ ]:
sar = ic.filter(ee.Filter.stringStartsWith('system:index', 'S1'))

In [ ]:
optical = ic.filter(ee.Filter.stringStartsWith('system:index', 'LC08'))

In [ ]:
start_time = "2019-05-13"
end_time = "2019-05-30"


In [ ]:
sar_water = sar.filterDate(start_time, end_time).sort('system:time_start').first()
optical_water = optical.filterDate(start_time, end_time).sort('system:time_start').first()

In [ ]:
sar_obs_date = sar_water.date().format('YYYY-MM-dd').getInfo()
optical_obs_date = optical_water.date().format('YYYY-MM-dd').getInfo()

In [ ]:
optical_water.getInfo() is None

In [ ]:
sentinel1_img = sentinel1.filter(ee.Filter.eq('system:index', sar_water.get('system:index'))).first()
landsat8_img = landsat8.filter(ee.Filter.eq('system:index', optical_water.get('system:index'))).first()

In [ ]:
fig,ax = plt.subplots(ncols=3,nrows=2,figsize=(15,7),
                      subplot_kw={'projection': ccrs.PlateCarree()})

dimsize = 1500

# use cartoee to get a map
cartoee.add_layer(ax[0,0], sentinel1_img, region=region, vis_params=s1_vis, dims=dimsize)
cartoee.add_layer(ax[0,1], sar_water, region=region, vis_params={"bands":'proba',"min":0,"max":100}, cmap='magma', dims=dimsize)
cartoee.add_layer(ax[0,2], sar_water, region=region, vis_params={"bands":'water',"min":0,"max":1}, cmap='Blues', dims=dimsize)


# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax[0,0], interval=[0.25, 0.25], linestyle=":")
cartoee.add_gridlines(ax[0,1], interval=[0.25, 0.25], linestyle=":")
cartoee.add_gridlines(ax[0,2], interval=[0.25, 0.25], linestyle=":")

# ax[0,0].xaxis.tick_top()
# ax[0,1].xaxis.tick_top()
# ax[0,2].xaxis.tick_top()
# ax[0,0].set_xticklabels([])
# ax[0,1].set_xticklabels([])
# ax[0,2].set_xticklabels([])
# ax[0,1].set_yticklabels([])
ax[0,1].get_yaxis().set_visible(False)
ax[0,2].yaxis.tick_right()

ax[0,0].set_title(f'a) Sentinel 1 observation {sar_obs_date}',loc='left', fontsize=10)
ax[0,1].set_title(f'b) Sentinel 1 water probability',loc='left', fontsize=10)
ax[0,2].set_title(f'c) Sentinel 1 predicted water',loc='left', fontsize=10)



# use cartoee to get a map
cartoee.add_layer(ax[1,0], landsat8_img, region=region, vis_params=lc8_vis, dims=dimsize)
cartoee.add_layer(ax[1,1], optical_water, region=region, vis_params={"bands":'proba',"min":0,"max":100}, cmap='magma', dims=dimsize)
cartoee.add_layer(ax[1,2], optical_water, region=region, vis_params={"bands":'water',"min":0,"max":1}, cmap='Blues', dims=dimsize)


# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax[1,0], interval=[0.25, 0.25], linestyle=":")
cartoee.add_gridlines(ax[1,1], interval=[0.25, 0.25], linestyle=":")
cartoee.add_gridlines(ax[1,2], interval=[0.25, 0.25], linestyle=":")

ax[1,1].get_yaxis().set_visible(False)
ax[1,2].yaxis.tick_right()


ax[1,0].set_title(f'd) Landsat 8 observation {optical_obs_date}',loc='left', fontsize=10)
ax[1,1].set_title(f'e) Landsat 8 water probability',loc='left', fontsize=10)
ax[1,2].set_title(f'f) Landsat 8 predicted water (gap filled)',loc='left', fontsize=10)


cax1 = fig.add_axes([0.41, 0.05, 0.2, 0.01])
cax2 = fig.add_axes([0.69, 0.05, 0.2, 0.01])

norm1 = mpl.colors.Normalize(vmin=0, vmax=100)

cb1 = mpl.colorbar.ColorbarBase(cax1, cmap=mpl.cm.magma,
                                norm=norm1,
                                orientation='horizontal')
cb1.set_label('Probability [%]')

bounds = [0.0, 0.5, 1.0]
cmap = mpl.colormaps['Blues']
cbarcolors = mpl.colors.ListedColormap([mpl.colors.rgb2hex(cmap(i)) for i in range(2)])
norm2 = mpl.colors.BoundaryNorm(bounds, cmap.N)

cb2 = mpl.colorbar.ColorbarBase(cax2, cmap=cmap,
                                norm=norm2,
                                orientation='horizontal')

cb2.ax.set_xticks([0.25,0.75])
cb2.ax.set_xticklabels(['No water','Water'])

plt.savefig('kh_sufacewater.png',dpi=300,bbox_inches='tight',facecolor='white')

plt.show()

In [ ]:
fig,ax = plt.subplots(ncols=2,nrows=1,figsize=(15,7),
                      subplot_kw={'projection': ccrs.PlateCarree()})

# use cartoee to get a map
cartoee.add_layer(ax[0], sentinel1_img, region=region, vis_params=s1_vis,cmap='gray', dims=500)

cartoee.add_layer(ax[1], sar_water, region=region, vis_params={"bands":'water',"min":0,"max":1}, cmap='Blues', dims=500)


# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax[0], interval=[0.25, 0.25], linestyle=":")
cartoee.add_gridlines(ax[1], interval=[0.25, 0.25], linestyle=":")

ax[0].set_title(f'Sentinel 1 observation {sar_obs_date}',loc='left')
ax[1].set_title(f'Sentinel 1 predicted water',loc='left')


# add coastlines using the cartopy api
# ax.coastlines(color="red")

plt.show()

# histograms of observation counts per region

In [ ]:
regions = ['Cambodia', 'Colombia','Gabon','Mexico','Myanmar','Zambia']

In [ ]:
abcs = ['a','b','c','d','e','f']

In [ ]:
f,ax = plt.subplots(nrows=3,ncols=2,sharex=False,figsize=(10,7))


histograms = {}

for i, region in enumerate(regions):
    val_collection = ee.FeatureCollection(f'projects/byu-hydroinformatics-gee/assets/kmarkert/datafusion/validation/{region}_all_wgs84')
    bbox = val_collection.geometry().bounds()

    ic = ee.ImageCollection(f'projects/byu-hydroinformatics-gee/assets/kmarkert/datafusion/{region}_data_fusion_v4') \
        .filterBounds(bbox)

    sar = ic.filter(ee.Filter.stringStartsWith('system:index', 'S1'))
    optical = ic.filter(ee.Filter.stringStartsWith('system:index', 'LC08'))

    sar_total_obs = sar.select('proba').count()
    optical_total_obs = optical.select('proba').count()
    all_total_obs = ic.select('proba').count()

    optical_x = optical_total_obs.reduceRegion(
        reducer=ee.Reducer.autoHistogram(),
        scale=30,
        geometry=bbox,
        maxPixels=1e10
    )
    sar_x = sar_total_obs.reduceRegion(
        reducer=ee.Reducer.autoHistogram(),
        scale=30,
        geometry=bbox,
        maxPixels=1e10
    )
    all_x = all_total_obs.reduceRegion(
        reducer=ee.Reducer.autoHistogram(),
        scale=30,
        geometry=bbox,
        maxPixels=1e10
    )

    # get the histogram data
    try:
        optical_hist = np.array(optical_x.getInfo()['proba'])

    except KeyError:
        optical_hist = np.array([[0,0]])

    sar_hist = np.array(sar_x.getInfo()['proba'])
    all_hist = np.array(all_x.getInfo()['proba'])

    # unpack the histogram data
    optical_bins, optical_counts = optical_hist[:,0], optical_hist[:,1]
    sar_bins, sar_counts = sar_hist[:,0], sar_hist[:,1]
    all_bins, all_counts = all_hist[:,0], all_hist[:,1]

    # normalize pixel counts
    optical_counts = optical_counts / np.sum(optical_counts)
    sar_counts = sar_counts / np.sum(sar_counts)
    all_counts = all_counts / np.sum(all_counts)

    r = i % 3
    c = 0 if i <= 2 else 1

    # ax[i].fill_between(optical_bins, 0, optical_counts, alpha=0.7,label='Optical only')
    # ax[i].fill_between(sar_bins, 0, sar_counts, alpha=0.7,color='C1',label='SAR only')
    # ax[i].fill_between(all_bins, 0, all_counts, alpha=0.7,color='C2', label='Optical+SAR')

    ax[r,c].bar(optical_bins, optical_counts, width=1,alpha=0.7,label='Optical only')
    ax[r,c].bar(sar_bins, sar_counts, width=1,alpha=0.7,color='C1',label='SAR only')
    if region != 'Myanmar':
        ax[r,c].bar(all_bins, all_counts, width=1,alpha=0.7,color='C2', label='Optical+SAR')


    if i == 3:
        ax[r,-1].legend(loc='upper right')

    ax[r,c].set_title(f'{abcs[i]}) {region}', loc='left', fontsize=10)


    histograms[region] = {
        'optical' : {
            'bins': optical_bins,
            'counts': optical_counts
        },
        'sar' : {
            'bins': sar_bins,
            'counts': sar_counts
        },
        'all': {
            'bins': all_bins,
            'counts': all_counts
        }

    }

# ax[-1].set_xlabel('Pixel observation count')

plt.tight_layout(rect=(0.03,0.03,1,1))

f.text(0.5,0.01, 'Pixel observation count', ha='center')
f.text(0.01,0.5, 'Frequency', va='center',rotation=90)

plt.savefig('region_observation_count_histograms.png',dpi=200,bbox_inches='tight')

plt.show()

In [ ]:
rows = []
for region, hists in histograms.items():
    print(region)
    region_row = {'region': region}
    for sensor, data in hists.items():
        x = np.sum(data['bins'] * data['counts'])
        print(f'{sensor} weighted average: {x}')
        region_row[sensor] = x
    print('\n')

    rows.append(region_row)


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(rows)

In [ ]:
df['optical_pct_change'] = (df['all'] - df['optical']) / df['optical']
df['sar_pct_change'] = (df['all'] - df['sar']) / df['sar']

In [ ]:
df

In [ ]:
df.describe()